# Cargar datos y librerías

In [33]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [44]:
df = pd.read_csv("../Datos/Weather_Spark/Clean/Panel_Weather_Spark_Bogota.csv")
df.tail()

,Hora,Fecha,Fecha_Hora,Estacion,Variable,Valor
1294663,19:00:00,2024-09-13,2024-09-13 19:00:00,Aeropuerto El Dorado,Nubosidad,Mayormente despejado
1294664,20:00:00,2024-09-13,2024-09-13 20:00:00,Aeropuerto El Dorado,Nubosidad,Parcialmente nublado
1294665,21:00:00,2024-09-13,2024-09-13 21:00:00,Aeropuerto El Dorado,Nubosidad,Parcialmente nublado
1294666,22:00:00,2024-09-13,2024-09-13 22:00:00,Aeropuerto El Dorado,Nubosidad,Parcialmente nublado
1294667,23:00:00,2024-09-13,2024-09-13 23:00:00,Aeropuerto El Dorado,Nubosidad,Parcialmente nublado


In [45]:
df2 = pd.read_excel("../Datos/Weather_Spark/Clean/Seccion_Cruzada_Weather_Spark_Bogota.xlsx")
df2

,Hora,Fecha,Fecha_Hora,Estacion,Temp.,Alt.,Vel_Viento,Dir_Viento,Vis.,Nubosidad
0,01:00:00,2000-01-01,2000-01-01 01:00:00,Aeropuerto El Dorado,11.7,1.024,7.20,O,10.0,Six oktas - 7/10 - 8/10
1,02:00:00,2000-01-01,2000-01-01 02:00:00,Aeropuerto El Dorado,11.0,1.024,0.00,O,11.3,Parcialmente nublado
2,03:00:00,2000-01-01,2000-01-01 03:00:00,Aeropuerto El Dorado,11.0,1.024,0.00,O,11.3,Parcialmente nublado
3,04:00:00,2000-01-01,2000-01-01 04:00:00,Aeropuerto El Dorado,11.0,1.024,0.00,O,12.0,Six oktas - 7/10 - 8/10
4,05:00:00,2000-01-01,2000-01-01 05:00:00,Aeropuerto El Dorado,11.0,1.024,0.00,O,12.0,Six oktas - 7/10 - 8/10
...,...,...,...,...,...,...,...,...,...,...
215773,19:00:00,2024-09-13,2024-09-13 19:00:00,Aeropuerto El Dorado,16.0,1.028,7.41,SE,10.0,Mayormente despejado
215774,20:00:00,2024-09-13,2024-09-13 20:00:00,Aeropuerto El Dorado,15.0,1.028,13.00,NE,10.0,Parcialmente nublado
215775,21:00:00,2024-09-13,2024-09-13 21:00:00,Aeropuerto El Dorado,14.0,1.029,7.41,NE,10.0,Parcialmente nublado
215776,22:00:00,2024-09-13,2024-09-13 22:00:00,Aeropuerto El Dorado,13.0,1.030,9.26,NE,10.0,Parcialmente nublado


# Ajustar variable de nubosidad

## Panel

In [46]:
nubosidad_df = df.loc[df["Variable"]=="Nubosidad"]

nubosidad_mapeo = {
    'No se detectaron nubes': 0,
    'Cielo y visibilidad bien': 0,
    'Pocas nubes': 1,
    'One okta - 1/10 or less but not zero': 1,
    'Mayormente despejado': 2,
    'Three oktas - 4/10': 3,
    'Five oktas - 6/10': 4,
    'Parcialmente nublado': 5,
    'Six oktas - 7/10 - 8/10': 6,
    'Mayormente nublado': 7,
    'Nublado': 8
}

# Asignar los valores numéricos a la columna "Valor"
nubosidad_df['Valor'] = nubosidad_df['Valor'].map(nubosidad_mapeo)

df.loc[df["Variable"] == "Nubosidad", "Valor"] = nubosidad_df["Valor"]
nubosidad_df = 0

df.tail(5)

,Hora,Fecha,Fecha_Hora,Estacion,Variable,Valor
1294663,19:00:00,2024-09-13,2024-09-13 19:00:00,Aeropuerto El Dorado,Nubosidad,2
1294664,20:00:00,2024-09-13,2024-09-13 20:00:00,Aeropuerto El Dorado,Nubosidad,5
1294665,21:00:00,2024-09-13,2024-09-13 21:00:00,Aeropuerto El Dorado,Nubosidad,5
1294666,22:00:00,2024-09-13,2024-09-13 22:00:00,Aeropuerto El Dorado,Nubosidad,5
1294667,23:00:00,2024-09-13,2024-09-13 23:00:00,Aeropuerto El Dorado,Nubosidad,5


## Sec cruzada

In [47]:
df2["Nubosidad"] = df2["Nubosidad"].map(nubosidad_mapeo)
df2.to_excel("../Datos/Weather_Spark/Clean/Seccion_Cruzada_Weather_Spark_Bogota_mapeado.xlsx", index=False)
df2=0

# Datos panel por año

In [50]:
df = df[["Fecha_Hora", "Variable", "Valor"]]

In [48]:
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Create a dictionary of dataframes, one for each year
dfs_by_year = {year: data for year, data in df.groupby(df['Fecha'].dt.year)}

# NOTA: Implementar el código de tal manera que solo un df por año sea cargado en memoria a la vez

# Save each dataframe to a separate sheet in an Excel file
with pd.ExcelWriter('../Datos/Weather_Spark/Clean/Panel_Weather_Spark_Por_Año_Bogota.xlsx') as writer:
    for year, data in dfs_by_year.items():
        data.to_excel(writer, sheet_name=str(year), index=False)